## Carga librerias necesarias


In [74]:
import pandas as pd 
import numpy as np
import os
from pandas import concat

## Carga del los dataset correspondientes al año 2022
Se cargan los csv de cada mes en una lista

In [75]:
file_names = os.listdir('../data/anio22/')
file_names

['abr_mo22.csv',
 'ago_mo22.csv',
 'dic_mo22.csv',
 'ene_mo22.csv',
 'feb_mo22.csv',
 'jul_mo22.csv',
 'jun_mo22.csv',
 'mar_mo22.csv',
 'may_mo22.csv',
 'nov_mo22.csv',
 'oct_mo22.csv',
 'sep_mo22.csv']

## Crear único dataset con los los datos del año 2022
Recorrer la lista files_names para crear un único dataset


In [76]:
df_aire = pd.DataFrame()
for file_name in file_names:
    df_current = pd.read_csv('../data/anio22/' + file_name, delimiter=';')
    print(file_name, df_current.shape)
    df_aire = pd.concat([df_aire, df_current], ignore_index=True)
    

abr_mo22.csv (3951, 56)
ago_mo22.csv (3960, 56)
dic_mo22.csv (4092, 56)
ene_mo22.csv (3929, 56)
feb_mo22.csv (3556, 56)
jul_mo22.csv (3968, 56)
jun_mo22.csv (3872, 56)
mar_mo22.csv (3946, 56)
may_mo22.csv (4059, 56)
nov_mo22.csv (3960, 56)
oct_mo22.csv (4092, 56)
sep_mo22.csv (3838, 56)


## Filtrar los datos con el componente NO2

In [77]:
df_aire = df_aire[(df_aire['MAGNITUD'] == 8)]
df_aire.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
90,28,79,4,8,28079004_8_8,2022,4,1,16.0,V,...,26.0,V,33.0,V,35.0,V,24.0,V,20.0,V
91,28,79,4,8,28079004_8_8,2022,4,2,13.0,V,...,18.0,V,23.0,V,24.0,V,18.0,V,15.0,V
92,28,79,4,8,28079004_8_8,2022,4,3,16.0,V,...,13.0,V,15.0,V,19.0,V,12.0,V,9.0,V
93,28,79,4,8,28079004_8_8,2022,4,4,8.0,V,...,28.0,V,28.0,V,29.0,V,32.0,V,19.0,V
94,28,79,4,8,28079004_8_8,2022,4,5,16.0,V,...,22.0,V,17.0,V,19.0,V,20.0,V,16.0,V


## Crear una nueva columna fecha 

In [78]:
df_aire['fecha'] = pd.to_datetime(df_aire[['ANO','MES','DIA']].rename(columns = {'ANO': 'YEAR', 'MES': 'MONTH', 'DIA': 'DAY'}))

## Extraer las datos de las medidas tomadas

In [79]:
data_no2_columns = [ 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24'] # feature
veri_columns = [ 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']

hora_column = np.arange(1,25) # feature label

## Crear un dataset auxiliar con los datos extraidos  y las nuevas columnas creadas

In [80]:


data_no2 = pd.DataFrame(columns=['hora', 'dato', 'verificado'])
aux = pd.DataFrame(columns=['ano', 'mes', 'dia', 'hora', 'dato', 'verificado', 'estacion']) # aux de próxima fila a ser añadida
for  (h,d,v) in zip(hora_column, data_no2_columns, veri_columns): 
     aux['dato'] = df_aire [ data_no2_columns[h-1] ] 
     aux['verificado'] = df_aire [ veri_columns[h-1] ] 
     aux['hora'] = h-1
     aux[['ano', 'mes', 'dia']] = df_aire[['ANO','MES','DIA']]
     aux['estacion'] = df_aire ['ESTACION']
     data_no2 = pd.concat([data_no2, aux], ignore_index=True)
data_no2 = data_no2.astype({'hora': 'int32'}, copy = False)

data_no2.head()



,hora,dato,verificado,ano,mes,dia,estacion
0,0,16.0,V,2022.0,4.0,1.0,4.0
1,0,13.0,V,2022.0,4.0,2.0,4.0
2,0,16.0,V,2022.0,4.0,3.0,4.0
3,0,8.0,V,2022.0,4.0,4.0,4.0
4,0,16.0,V,2022.0,4.0,5.0,4.0


## Añadir la columna fecha en formato datetime al nuevo dataset 

In [81]:
data_no2['fecha'] = pd.to_datetime(data_no2[['ano','mes','dia','hora']].rename(columns = {'ano': 'year', 'mes': 'month','dia': 'day', 'hora': 'hour'}))
data_no2.head()

,hora,dato,verificado,ano,mes,dia,estacion,fecha
0,0,16.0,V,2022.0,4.0,1.0,4.0,2022-04-01
1,0,13.0,V,2022.0,4.0,2.0,4.0,2022-04-02
2,0,16.0,V,2022.0,4.0,3.0,4.0,2022-04-03
3,0,8.0,V,2022.0,4.0,4.0,4.0,2022-04-04
4,0,16.0,V,2022.0,4.0,5.0,4.0,2022-04-05


## Multi-index data frame con estacion y fecha :O

In [82]:
data_no2.set_index(['estacion', 'fecha'], inplace=True)
data_no2.head()

hora  dato verificado     ano  mes  dia
estacion fecha                                              
4.0      2022-04-01     0  16.0          V  2022.0  4.0  1.0
         2022-04-02     0  13.0          V  2022.0  4.0  2.0
         2022-04-03     0  16.0          V  2022.0  4.0  3.0
         2022-04-04     0   8.0          V  2022.0  4.0  4.0
         2022-04-05     0  16.0          V  2022.0  4.0  5.0

## Ordenar por estación y fecha 

In [83]:
data_no2 = data_no2.sort_values(['estacion', 'fecha'])
data_no2.head()



hora  dato verificado     ano  mes   dia
estacion fecha                                                        
4.0      2022-03-25 00:00:00     0   0.0          N  2022.0  3.0  25.0
         2022-03-25 01:00:00     1   0.0          N  2022.0  3.0  25.0
         2022-03-25 02:00:00     2   0.0          N  2022.0  3.0  25.0
         2022-03-25 03:00:00     3   0.0          N  2022.0  3.0  25.0
         2022-03-25 04:00:00     4   0.0          N  2022.0  3.0  25.0

## Rellenar con NaN las filas perdidas 
 Para cada estación, rellenamos con NaN si hay alguna fila perdida y guardamos el resultado en un nuevo dataframe que ya debería tener todas las filas


In [84]:
estaciones = data_no2.index.get_level_values('estacion').unique()
datetime_index = pd.date_range(start='1/1/2022', end='12/31/2022 23:00:00', freq='H', name='fecha')
data_no2_nuevo = pd.DataFrame()
for estacion in estaciones:
    #data_no2_estacion = data_no2[data_no2['estacion'] == estacion]
    data_no2_estacion = data_no2.loc[estacion]
    print(estacion, len(data_no2_estacion))
    data_no2_estacion = data_no2_estacion.reindex(datetime_index)
    data_no2_estacion.reset_index(inplace=True)
    print(estacion, len(data_no2_estacion))
    data_no2_estacion['estacion'] = estacion
    data_no2_nuevo = pd.concat([data_no2_nuevo, data_no2_estacion],ignore_index=True)
    
print(len(data_no2_nuevo))

4.0 6720
4.0 8760
8.0 8760
8.0 8760
11.0 8760
11.0 8760
16.0 6336
16.0 8760
17.0 8760
17.0 8760
18.0 8760
18.0 8760
24.0 8736
24.0 8760
27.0 8760
27.0 8760
35.0 8760
35.0 8760
36.0 8520
36.0 8760
38.0 8760
38.0 8760
39.0 8760
39.0 8760
40.0 8760
40.0 8760
47.0 8712
47.0 8760
48.0 8760
48.0 8760
49.0 8760
49.0 8760
50.0 8760
50.0 8760
54.0 8760
54.0 8760
55.0 8760
55.0 8760
56.0 8760
56.0 8760
57.0 8760
57.0 8760
58.0 8760
58.0 8760
59.0 8760
59.0 8760
60.0 8760
60.0 8760
210240


In [85]:
# Multi-index data_no2 frame con estacion y fecha :O
data_no2_nuevo.set_index(['estacion', 'fecha'], inplace=True)

##  Actualizar  data con el nuevo df:

In [86]:
data_no2 = data_no2_nuevo
data_no2.info()
data_no2

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 210240 entries, (4.0, Timestamp('2022-01-01 00:00:00')) to (60.0, Timestamp('2022-12-31 23:00:00'))
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   hora        205464 non-null  float64
 1   dato        205464 non-null  float64
 2   verificado  205464 non-null  object 
 3   ano         205464 non-null  float64
 4   mes         205464 non-null  float64
 5   dia         205464 non-null  float64
dtypes: float64(5), object(1)
memory usage: 10.5+ MB


hora  dato verificado     ano   mes   dia
estacion fecha                                                         
4.0      2022-01-01 00:00:00   NaN   NaN        NaN     NaN   NaN   NaN
         2022-01-01 01:00:00   NaN   NaN        NaN     NaN   NaN   NaN
         2022-01-01 02:00:00   NaN   NaN        NaN     NaN   NaN   NaN
         2022-01-01 03:00:00   NaN   NaN        NaN     NaN   NaN   NaN
         2022-01-01 04:00:00   NaN   NaN        NaN     NaN   NaN   NaN
...                            ...   ...        ...     ...   ...   ...
60.0     2022-12-31 19:00:00  19.0  60.0          V  2022.0  12.0  31.0
         2022-12-31 20:00:00  20.0  68.0          V  2022.0  12.0  31.0
         2022-12-31 21:00:00  21.0  67.0          V  2022.0  12.0  31.0
         2022-12-31 22:00:00  22.0  58.0          V  2022.0  12.0  31.0
         2022-12-31 23:00:00  23.0  49.0          V  2022.0  12.0  31.0

[210240 rows x 6 columns]

In [87]:
# Mostramos los datos que se habían perdidos
missing_data_no2 = data_no2[data_no2['dato'].isna()]
print(f'Cantidad de datos no existentes (perdidos): {len(missing_data_no2)}')
missing_data_no2

Cantidad de datos no existentes (perdidos): 4776


hora  dato verificado  ano  mes  dia
estacion fecha                                                    
4.0      2022-01-01 00:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-01-01 01:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-01-01 02:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-01-01 03:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-01-01 04:00:00   NaN   NaN        NaN  NaN  NaN  NaN
...                            ...   ...        ...  ...  ...  ...
47.0     2022-03-13 19:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-03-13 20:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-03-13 21:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-03-13 22:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-03-13 23:00:00   NaN   NaN        NaN  NaN  NaN  NaN

[4776 rows x 6 columns]

In [88]:
# Rellenamos las columnas conciernientes a la fecha que faltan en esos datos perdidos excepto la columna 'dato' que ya trataremos más adelante
dt = missing_data_no2.index.get_level_values('fecha')
data_no2.loc[data_no2['dato'].isna(),'ano'] = dt.year
data_no2.loc[data_no2['dato'].isna(),'mes'] = dt.month
data_no2.loc[data_no2['dato'].isna(),'dia'] = dt.day
data_no2.loc[data_no2['dato'].isna(),'hora'] = dt.hour
data_no2 = data_no2.astype({'hora': 'int32'}, copy = False)

#data_no2.loc[data_no2['dato'].isna(),['ano', 'mes', 'dia', 'hora']] = [dt.year, dt.month, dt.day, dt.hour]
data_no2[data_no2['dato'].isna()]

hora  dato verificado     ano  mes   dia
estacion fecha                                                        
4.0      2022-01-01 00:00:00     0   NaN        NaN  2022.0  1.0   1.0
         2022-01-01 01:00:00     1   NaN        NaN  2022.0  1.0   1.0
         2022-01-01 02:00:00     2   NaN        NaN  2022.0  1.0   1.0
         2022-01-01 03:00:00     3   NaN        NaN  2022.0  1.0   1.0
         2022-01-01 04:00:00     4   NaN        NaN  2022.0  1.0   1.0
...                            ...   ...        ...     ...  ...   ...
47.0     2022-03-13 19:00:00    19   NaN        NaN  2022.0  3.0  13.0
         2022-03-13 20:00:00    20   NaN        NaN  2022.0  3.0  13.0
         2022-03-13 21:00:00    21   NaN        NaN  2022.0  3.0  13.0
         2022-03-13 22:00:00    22   NaN        NaN  2022.0  3.0  13.0
         2022-03-13 23:00:00    23   NaN        NaN  2022.0  3.0  13.0

[4776 rows x 6 columns]

In [89]:


# 6. Por último, marcamos los datos no verificados como NaN
no_verificados = data_no2.loc[data_no2['verificado'] != 'V']
print(f'Cantidad de datos existentes pero no válidos:{len(no_verificados)}')
data_no2.loc[data_no2['verificado'] != 'V', ['dato']] = np.nan
data_no2.loc[data_no2['verificado'] != 'V']



Cantidad de datos existentes pero no válidos:6642


hora  dato verificado     ano   mes   dia
estacion fecha                                                         
4.0      2022-01-01 00:00:00     0   NaN        NaN  2022.0   1.0   1.0
         2022-01-01 01:00:00     1   NaN        NaN  2022.0   1.0   1.0
         2022-01-01 02:00:00     2   NaN        NaN  2022.0   1.0   1.0
         2022-01-01 03:00:00     3   NaN        NaN  2022.0   1.0   1.0
         2022-01-01 04:00:00     4   NaN        NaN  2022.0   1.0   1.0
...                            ...   ...        ...     ...   ...   ...
60.0     2022-11-21 12:00:00    12   NaN          N  2022.0  11.0  21.0
         2022-11-30 09:00:00     9   NaN          N  2022.0  11.0  30.0
         2022-11-30 10:00:00    10   NaN          N  2022.0  11.0  30.0
         2022-12-21 11:00:00    11   NaN          N  2022.0  12.0  21.0
         2022-12-21 12:00:00    12   NaN          N  2022.0  12.0  21.0

[6642 rows x 6 columns]

In [90]:
# borramos columna verificado
del data_no2['verificado']
data_no2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 210240 entries, (4.0, Timestamp('2022-01-01 00:00:00')) to (60.0, Timestamp('2022-12-31 23:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   hora    210240 non-null  int32  
 1   dato    203598 non-null  float64
 2   ano     210240 non-null  float64
 3   mes     210240 non-null  float64
 4   dia     210240 non-null  float64
dtypes: float64(4), int32(1)
memory usage: 8.1 MB


In [91]:


# Primero de todo, deshacemos el multíndice
data_no2.reset_index(inplace=True) # Salvar los datos sin indices de estacion y fecha



In [92]:


# 7. Añadimos columna para el día de la semana
#dates = data_no2.index.get_level_values('fecha').to_pydatetime() # con multiindex
dates = data_no2['fecha'].dt.to_pydatetime() # con column y sin multiindex
days_of_week = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
data_no2['dia_semana'] = np.array([days_of_week[d.weekday()] for d in dates])



In [93]:
data_no2 = data_no2.rename(columns={'dato': 'NO2'}, copy = False)

In [94]:


data_no2.info()
data_no2.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210240 entries, 0 to 210239
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   estacion    210240 non-null  float64       
 1   fecha       210240 non-null  datetime64[ns]
 2   hora        210240 non-null  int32         
 3   NO2         203598 non-null  float64       
 4   ano         210240 non-null  float64       
 5   mes         210240 non-null  float64       
 6   dia         210240 non-null  float64       
 7   dia_semana  210240 non-null  object        
dtypes: datetime64[ns](1), float64(5), int32(1), object(1)
memory usage: 12.0+ MB


,estacion,fecha,hora,NO2,ano,mes,dia,dia_semana
0,4.0,2022-01-01 00:00:00,0,NaN,2022.0,1.0,1.0,Sabado
1,4.0,2022-01-01 01:00:00,1,NaN,2022.0,1.0,1.0,Sabado
2,4.0,2022-01-01 02:00:00,2,NaN,2022.0,1.0,1.0,Sabado
3,4.0,2022-01-01 03:00:00,3,NaN,2022.0,1.0,1.0,Sabado
4,4.0,2022-01-01 04:00:00,4,NaN,2022.0,1.0,1.0,Sabado


In [95]:
data_no2.tail()

,estacion,fecha,hora,NO2,ano,mes,dia,dia_semana
210235,60.0,2022-12-31 19:00:00,19,60.0,2022.0,12.0,31.0,Sabado
210236,60.0,2022-12-31 20:00:00,20,68.0,2022.0,12.0,31.0,Sabado
210237,60.0,2022-12-31 21:00:00,21,67.0,2022.0,12.0,31.0,Sabado
210238,60.0,2022-12-31 22:00:00,22,58.0,2022.0,12.0,31.0,Sabado
210239,60.0,2022-12-31 23:00:00,23,49.0,2022.0,12.0,31.0,Sabado


In [96]:
# Serie con solamente los datos de contaminación
datos = data_no2['NO2']
datos

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
210235    60.0
210236    68.0
210237    67.0
210238    58.0
210239    49.0
Name: NO2, Length: 210240, dtype: float64

In [97]:
#datos.to_csv('data_no2_hist.csv') # Salvado serie solo datos históricos
data_no2.to_csv('../data/aire/data_n02_fecha.csv') # Salvado con los datos de las fechas